# **Reinforcement Learning Project - Inventory beer game**

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# helperfunctions
# function to convert real states to coded states
def coded_state(inventory):
    if inventory < -8:
        return 1
    elif inventory < 0:
        return 2
    elif inventory < 8:
        return 3
    elif inventory < 16:
        return 4
    else:
        return 5

# function to view episodes
def fun_episode(S, CS, D, O, x, y, r):
    time_steps = ['t='+str(i) for i in np.arange(start=-1, stop=T+1)]

    df = pd.DataFrame({'Inventory/States S': [(0, 0)] + [tuple(i) for i in S],
                   'Coded states CS': [(0, 0)] + [tuple(i) for i in CS],
                   'Demand x': [tuple(i) for i in x] + [(0, 0)],
                   'Distribution amount D': [(0, 0)] + [tuple(i) for i in D],
                   'Action y': [(0, 0)] + [tuple(i) for i in y],
                   'Ordering size': [(0, 0)] + [tuple(i) for i in O],
                   'Costs r': [(0, 0)] + [tuple(i) for i in r]},
                   index=time_steps)
    df.index.name = 'Time'
    return df

In [3]:
# Setup
# Supply Chain and its agents
supply_chain = {'level 0': 'Customer',
                'level 1': 'Retailer',
                'level 2': 'Distributor'}

agents = [supply_chain[i] for i in list(supply_chain.keys())[1:]]


# All possible coded inventory positions of agents and the respective state pairs (5 states with 25 state pairs)
states = np.arange(start=1, stop=6)
state_pairs = [(i, j) for i in states for j in states]

# y-value in ordering rule x+y with x equal to the demand from the downstream agent and the respective action pairs (4 actions with 16 action pairs)
actions = np.arange(stop=4)
action_pairs = [(i, j) for i in actions for j in actions]


# Initial matrix with Q-values (minimization -> high values)
Q_values = np.full(shape=(len(state_pairs), len(action_pairs)), fill_value=500)
df_Q_values = pd.DataFrame(data=Q_values, index=state_pairs, columns=action_pairs)
df_Q_values.head()

,"(0, 0)","(0, 1)","(0, 2)","(0, 3)","(1, 0)","(1, 1)","(1, 2)","(1, 3)","(2, 0)","(2, 1)","(2, 2)","(2, 3)","(3, 0)","(3, 1)","(3, 2)","(3, 3)"
"(1, 1)",500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500
"(1, 2)",500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500
"(1, 3)",500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500
"(1, 4)",500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500
"(1, 5)",500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500


In [4]:
# Define parameters
iteration = 1
max_iteration = 2
t = 0
T = 5
gamma = 1
alpha = 0.17
proba_exploitation = 0.02

# Set up Lists to store the results of each iteration/episode
S = [list(np.repeat(12, len(agents)))] + [list(np.repeat(0, len(agents))) for i in range((T))] #(start-)inventory positions/states per agent
CS = [list(np.repeat(coded_state(S[0][0]), len(agents)))] + [list(np.repeat(0, len(agents))) for i in range((T))] #coded states per agent
O = [list(np.repeat(4, len(agents)))] + [list(np.repeat(0, len(agents))) for i in range(T)] #ordering size per agent
D = [list(np.repeat(0, len(agents))) for i in range(T+1)] #distribution amount per agent
x = [list(np.repeat(0, len(agents))) for i in range(T+1)] #demand from downstream level per agent
y = [list(np.repeat(0, len(agents))) for i in range(T+1)] #action per agent
r = [list(np.repeat(0, len(agents))) for i in range(T+1)] #reward per agent

R = [0 for i in range(T)] #reward of the supply chain per time step t
G = [0 for i in range(T)] #return of the supply chian: total discounted rewards

while iteration < max_iteration:
    print('------------------episode {}------------------'.format(iteration))

    while t < T:
        print('-----------------time step t={}----------------'.format(t))

        # print state and coded state
        state = tuple(S[t])
        c_state = tuple(CS[t])
        print('State: {} and coded state: {}'.format(state, c_state))

        for agent in range(len(agents)):
            level = supply_chain['level ' + str(agent+1)]
            print(level + ':')

        # step 1: previous orders are received from the upstream agent
            inventory = S[t][agent] + O[t-1][agent]
            print('Inventory size: {}'.format(inventory))

        # step 2: receive the new demand of the downstream agent
            if level == 'Retailer':
                x[t][agent] = np.random.randint(low=0, high=15)
            else: x[t][agent] = O[t-1][agent-1]
            print('Demand from downstream ({}) at t-1: {}'.format(supply_chain['level ' + str(agent)], x[t][agent]))

        # step 3: fulfill order of downstream agent from onhand inventory and calculate possible backlog costs
            D[t][agent] = min(x[t][agent], inventory) #distribution quantity
            #S[t+1][agent] = S[t][agent] - x[t][agent] #state of next time step
            backlog = x[t][agent] - D[t][agent] #penalty/backlog costs
            print('Distribution size: {}'.format(D[t][agent]))
            print('Backlog size: {}'.format(backlog))

        # step 4: placing order for stock replenishment
            # define best action for agent and select it with initially very small probability (first exploration and mostly random choices)
            best_action = df_Q_values.iloc[df_Q_values.index.get_loc(tuple(CS[t]))].idxmin()[agent]
            print('Best action: {}'.format(best_action))

            y[t][agent] = random.choices([best_action] + list(actions), weights=[proba_exploitation] + list(np.repeat((1 - proba_exploitation) / len(actions), len(actions))))[0]
            print('Action: {}'.format(y[t][agent]))

            O[t][agent] = x[t][agent] + y[t][agent]
            print('Ordering size: {}'.format(O[t][agent]))

            # calculate agent's costs (onhand inventory holding costs + penalty costs)
            r[t][agent] = 1 * (inventory - D[t][agent]) + 2 * backlog
            print('Costs: {}'.format(r[t][agent]))

            if level != agents[-1]: print('------------------next agent------------------')
            else: print()

        # calculate the total supply chain costs in t
        action = tuple(y[t])
        costs = np.sum(r[t])
        print('Supply Chain costs in state {} with action {} at t={}: {}'.format(state, action, t, costs))

        # update states for t+1
        for agent in range(len(agents)):
            S[t+1][agent] = S[t][agent] + O[t-1][agent] - D[t][agent] #inventory - distribution size
            CS[t+1][agent] = coded_state(S[t+1][agent])
        
        # increase time step t
        t += 1
        print('\n\n')

    # view last episode
    display(fun_episode(S, CS, D, O, x, y, r))

    # update Q-values
    # Loop from T to t=0 to calculate immediate rewards and returns
    for t in np.arange(start=0, stop=T)[::-1]:
        # calculate the immediate reward of each visited state-action-pair
        R[t] = np.sum(r[t])
        #print('Immediate reward r in t={}: {}'.format(t, R[t]))

        # calculate the return G: total discounted rewards
        G[t] = R[t] + np.sum(R[t+1:] * np.array([gamma**i for i in np.arange(start=1, stop=T-t)]))
        
        #print()

    #print(R)
    print(G)

    # Loop to update the Q-values
    for t in range(T):
        state = tuple(CS[t])
        action = tuple(y[t])

        # get current Q-value
        q_value = df_Q_values.iloc[df_Q_values.index.get_loc(state), df_Q_values.columns.get_loc(action)]
        print('t={}: state: {}, action: {}, old Q-value: {}, return: {}'.format(t, state, action, q_value, G[t]))

        # update Q-values according to equation 12 in the paper (slide 30 TD-learning script)
        print((1-alpha) * q_value + alpha * G[t])
        df_Q_values.iloc[df_Q_values.index.get_loc(state), df_Q_values.columns.get_loc(action)] = (1-alpha) * q_value + alpha * G[t] #equal to: q_value + alpha * (G[t] - q_value)
        print()

    display(df_Q_values)

    # increase exploitation probability
    proba_exploitation += 0.05
    #print('\n\n')

    # start with next iteration
    iteration += 1

------------------episode 1------------------
-----------------time step t=0----------------
State: (12, 12) and coded state: (4, 4)
Retailer:
Inventory size: 12
Demand from downstream (Customer) at t-1: 12
Distribution size: 12
Backlog size: 0
Best action: 0
Action: 1
Ordering size: 13
Costs: 0
------------------next agent------------------
Distributor:
Inventory size: 12
Demand from downstream (Retailer) at t-1: 0
Distribution size: 0
Backlog size: 0
Best action: 0
Action: 2
Ordering size: 2
Costs: 12

Supply Chain costs in state (12, 12) with action (1, 2) at t=0: 12



-----------------time step t=1----------------
State: (0, 12) and coded state: (3, 4)
Retailer:
Inventory size: 13
Demand from downstream (Customer) at t-1: 2
Distribution size: 2
Backlog size: 0
Best action: 0
Action: 0
Ordering size: 2
Costs: 11
------------------next agent------------------
Distributor:
Inventory size: 14
Demand from downstream (Retailer) at t-1: 13
Distribution size: 13
Backlog size: 0
Best actio

,Inventory/States S,Coded states CS,Demand x,Distribution amount D,Action y,Ordering size,Costs r
Time,,,,,,,
t=-1,"(0, 0)","(0, 0)","(12, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
t=0,"(12, 12)","(4, 4)","(2, 13)","(12, 0)","(1, 2)","(13, 2)","(0, 12)"
t=1,"(0, 12)","(3, 4)","(1, 2)","(2, 13)","(0, 0)","(2, 13)","(11, 1)"
t=2,"(11, 1)","(4, 3)","(1, 2)","(1, 2)","(1, 0)","(2, 2)","(12, 12)"
t=3,"(12, 12)","(4, 4)","(11, 4)","(1, 2)","(3, 0)","(4, 2)","(13, 12)"
t=4,"(13, 12)","(4, 4)","(0, 0)","(11, 4)","(2, 1)","(13, 5)","(6, 10)"
t=5,"(6, 10)","(3, 4)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"


[89, 77, 65, 41, 16.0]
t=0: state: (4, 4), action: (1, 2), old Q-value: 500, return: 89
430.13

t=1: state: (3, 4), action: (0, 0), old Q-value: 500, return: 77
428.09

t=2: state: (4, 3), action: (1, 0), old Q-value: 500, return: 65
426.05

t=3: state: (4, 4), action: (3, 0), old Q-value: 500, return: 41
421.97

t=4: state: (4, 4), action: (2, 1), old Q-value: 500, return: 16.0
417.72



,"(0, 0)","(0, 1)","(0, 2)","(0, 3)","(1, 0)","(1, 1)","(1, 2)","(1, 3)","(2, 0)","(2, 1)","(2, 2)","(2, 3)","(3, 0)","(3, 1)","(3, 2)","(3, 3)"
"(1, 1)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(1, 2)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(1, 3)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(1, 4)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(1, 5)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(2, 1)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(2, 2)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(2, 3)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(2, 4)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500
"(2, 5)",500.00,500,500,500,500.00,500,500.00,500,500,500.00,500,500,500.00,500,500,500


In [5]:
# Loop from T to t=0 to calculate immediate rewards and returns
for t in np.arange(start=0, stop=T)[::-1]:
    # calculate the immediate reward of each visited state-action-pair
    R[t] = np.sum(r[t])
    #print('Immediate reward r in t={}: {}'.format(t, R[t]))

    # calculate the return G: total discounted rewards
    G[t] = R[t] + np.sum(R[t+1:] * np.array([gamma**i for i in np.arange(start=1, stop=T-t)]))
     
    #print()

#print(R)
print(G)

# Loop to update the Q-values
for t in range(T):
    state = tuple(CS[t])
    action = tuple(y[t])

    # get current Q-value
    q_value = df_Q_values.iloc[df_Q_values.index.get_loc(state), df_Q_values.columns.get_loc(action)]
    print('t={}: state: {}, action: {}, old Q-value: {}, return: {}'.format(t, state, action, q_value, G[t]))

    # update Q-values according to equation 12 in the paper (slide 30 TD-learning script)
    print((1-alpha) * q_value + alpha * G[t])
    df_Q_values.iloc[df_Q_values.index.get_loc(state), df_Q_values.columns.get_loc(action)] = (1-alpha) * q_value + alpha * G[t] #equal to: q_value + alpha * (G[t] - q_value)
    print()

[89, 77, 65, 41, 16.0]
t=0: state: (4, 4), action: (1, 2), old Q-value: 430.13, return: 89
372.1379

t=1: state: (3, 4), action: (0, 0), old Q-value: 428.09, return: 77
368.40469999999993

t=2: state: (4, 3), action: (1, 0), old Q-value: 426.05, return: 65
364.6715

t=3: state: (4, 4), action: (3, 0), old Q-value: 421.97, return: 41
357.2051

t=4: state: (4, 4), action: (2, 1), old Q-value: 417.72, return: 16.0
349.42760000000004



In [6]:
df_Q_values

,"(0, 0)","(0, 1)","(0, 2)","(0, 3)","(1, 0)","(1, 1)","(1, 2)","(1, 3)","(2, 0)","(2, 1)","(2, 2)","(2, 3)","(3, 0)","(3, 1)","(3, 2)","(3, 3)"
"(1, 1)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(1, 2)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(1, 3)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(1, 4)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(1, 5)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(2, 1)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(2, 2)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(2, 3)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(2, 4)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500
"(2, 5)",500.0000,500,500,500,500.0000,500,500.0000,500,500,500.0000,500,500,500.0000,500,500,500


### **Test performance of the Reinforcement learning ordering mechanism (RLOM) on main and test problems**

In [7]:
# Main test problem
customer_demand = [15,10,8,14,9,3,13,2,13,11,3,4,6,11,15,12,15,4,12,3,13,10,15,15,3,11,1,13,10,10,0,0,8,0,14]
lead_times = [2,0,2,4,4,4,0,2,4,1,1,0,0,1,1,0,1,1,2,1,1,1,4,2,2,1,4,3,4,1,4,0,3,3,4]

# Test problem 1
customer_demand = [5,14,14,13,2,9,5,9,14,14,12,7,5,1,13,3,12,4,0,15,11,10,6,0,6,6,5,11,8,4,4,12,13,8,12]
lead_times = [2,0,2,4,4,4,0,2,4,1,1,0,0,1,1,0,1,1,2,1,1,1,4,2,2,1,4,3,4,1,4,0,3,3,4]

# Test problem 2
customer_demand = [15,10,8,14,9,3,13,2,13,11,3,4,6,11,15,12,15,4,12,3,13,10,15,15,3,11,1,13,10,10,0,0,8,0,14]
lead_times = [4,2,2,0,2,2,1,1,3,0,0,3,3,3,4,1,1,1,3,0,4,2,3,4,1,3,3,3,0,3,4,3,3,0,3]

# Test problem 3
customer_demand = [13,13,12,10,14,13,13,10,2,12,11,9,11,3,7,6,12,12,3,10,3,9,4,15,12,7,15,5,1,15,11,9,14,0,4]
lead_times = [4,2,2,0,2,2,1,1,3,0,0,3,3,3,4,1,1,1,3,0,4,2,3,4,1,3,3,3,0,3,4,3,3,0,3]

# Own test problem
customer_demand = list(np.random.randint(low=0, high=15, size=35))
lead_times = list(np.random.randint(low=0, high=4, size=35))